In [1]:
import os
import re

import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
tqdm.pandas()

from shapely.geometry import MultiPolygon, Polygon
from shapely.validation import make_valid

In [2]:
from constants import CEN_YEARS
from constants import YEAR_CODES

mpoly_GTA = gpd.read_file('../data/geo/regions/GTA.gpkg').geometry.to_list()[0]

Prune geometries to select for the GTA-only versions

In [6]:
cen_years_81 = [y for y in CEN_YEARS if y >= 1981]  # Prior years do not have CSD data

for year in tqdm(cen_years_81):
    gdf_csd_all = gpd.read_file(f'../data/geo/{year}_csd/csd_{year}.geojson')
    gdf_csd_all['gta_overlap'] = gdf_csd_all.apply(lambda row: row.geometry.intersection(mpoly_GTA).area / row.geometry.area, axis=1)
    gdf_csd_gta = gdf_csd_all[gdf_csd_all.gta_overlap >= 0.25]
    gdf_csd_gta.to_file(f'../data/geo/{year}_csd/csd_gta_{year}.gpkg', driver="GPKG")

100%|██████████| 9/9 [02:19<00:00, 15.47s/it]


In [10]:
# For each year, save those CT's which are in the GTA
for year in tqdm(CEN_YEARS):
    gdf_ct_all = gpd.read_file(f"../data/geo/{year}_ct/ct_{year}.zip")
    gdf_ct_all['gta_overlap'] = gdf_ct_all.apply(lambda row: row.geometry.intersection(mpoly_GTA).area / row.geometry.area, axis=1)
    gdf_ct_gta = gdf_ct_all[gdf_ct_all.gta_overlap > 0.5]
    gdf_ct_gta.to_file(f"../data/geo/{year}_ct/ct_gta_{year}.gpkg", driver='GPKG')

100%|██████████| 15/15 [00:42<00:00,  2.82s/it]


Join together CSD/CT boundaries and data, then create a joint geographical dataset which covers the full GTA by CSD's where CT's are missing weighted proportionately by how much they are included.

In the case of years without CSD data (<1981), join the CSD geometry from 2021 with NaN values.

In [3]:
def add_census_values_to_gdf(gdf_full, gdf_small, cen_year, cen_var):
    cols = YEAR_CODES[cen_year][cen_var]

    if len(cols) == 0:  
        gdf_small[cen_var] = np.nan
        if cen_var == 'num_not_vm_tot':
            gdf_small.rename(columns={'num_not_vm_tot': 'num_vm_tot'}, inplace=True)
    elif len(cols) > 1:  # New immigrants has multiple tags
        gdf_small.loc[:, cen_var] = gdf_full.loc[:, cols].sum(axis=1)
    elif cen_var == 'num_not_vm_tot':  # Need to compute total - NOT
        pop_total = gdf_full[YEAR_CODES[cen_year]['num_pop_tot'][0]]
        non_vm_total = gdf_full[cols[0]]  # Use [0] to get string from list
        gdf_small['num_vm_tot'] = pop_total - non_vm_total
    else:
        gdf_small.loc[:, cen_var] = gdf_full.loc[:, cols]

In [4]:
# Obtain geography for 2021 CSD data in the GTA
gdf_csd_gta_2021 = gpd.read_file(f"../data/geo/2021_csd/csd_gta_2021.gpkg")
gdf_csd_gta_2021 = gdf_csd_gta_2021[['geosid', 'geometry']]
gdf_csd_gta_2021['geosid'] = gdf_csd_gta_2021['geosid'].astype(str)
gdf_csd_gta_2021['num_pop_tot'] = np.nan
gdf_csd_gta_2021['num_imm_tot'] = np.nan

In [16]:
for cen_year in tqdm(CEN_YEARS):
    if (cen_year < 1961) or (cen_year == 1966) or (cen_year == 1976):
        continue

    gdf_ct_gta = gpd.read_file(f"../data/geo/{cen_year}_ct/ct_gta_{cen_year}.gpkg")
    df_ct_cen = pd.read_csv(f"../data/census/{cen_year}_ct_wide/census_wide_{cen_year}_ct.csv")

    gdf_ct_gta['geosid'] = gdf_ct_gta['geosid'].astype(str)
    df_ct_cen['geosid'] = df_ct_cen['geosid'].astype(str)
    df_ct_cen['geosid'] = df_ct_cen['geosid'].apply(lambda x: x[:-2] + '.00' if x.endswith('.0') else x)

    gdf_ct_full = pd.merge(
        gdf_ct_gta,
        df_ct_cen,
        on='geosid',
        how='left'
    )

    gdf_ct_small = gdf_ct_full[['geosid', 'geometry']].copy()
    add_census_values_to_gdf(gdf_ct_full, gdf_ct_small, cen_year, 'num_pop_tot')
    add_census_values_to_gdf(gdf_ct_full, gdf_ct_small, cen_year, 'num_imm_tot')
    add_census_values_to_gdf(gdf_ct_full, gdf_ct_small, cen_year, 'num_imm_new')
    add_census_values_to_gdf(gdf_ct_full, gdf_ct_small, cen_year, 'num_imm_2nd_tot')
    add_census_values_to_gdf(gdf_ct_full, gdf_ct_small, cen_year, 'num_not_vm_tot')

    if cen_year >= 1981:
        gdf_csd_gta = gpd.read_file(f"../data/geo/{cen_year}_csd/csd_gta_{cen_year}.gpkg")
        df_csd_cen = pd.read_csv(f"../data/census/{cen_year}_csd_wide/census_wide_{cen_year}_csd.csv")
        
        gdf_csd_gta['geosid'] = gdf_csd_gta['geosid'].astype(str)
        df_csd_cen['geosid'] = df_csd_cen['geosid'].astype(str)

        gdf_csd_full = pd.merge(
            gdf_csd_gta,
            df_csd_cen,
            on='geosid',
            how='left'
        )

        gdf_csd_small = gdf_csd_full[['geosid', 'geometry']].copy()
        add_census_values_to_gdf(gdf_csd_full, gdf_csd_small, cen_year, 'num_pop_tot')
        add_census_values_to_gdf(gdf_csd_full, gdf_csd_small, cen_year, 'num_imm_tot')
        add_census_values_to_gdf(gdf_csd_full, gdf_csd_small, cen_year, 'num_imm_new')
        add_census_values_to_gdf(gdf_csd_full, gdf_csd_small, cen_year, 'num_imm_2nd_tot')
        add_census_values_to_gdf(gdf_csd_full, gdf_csd_small, cen_year, 'num_not_vm_tot')

    ## Join together CTs and CSDs
    # Project geometries to CRS 3857 for accurate area calculations
    gdf_ct_small_proj = gdf_ct_small.to_crs(3857)
    if cen_year >= 1981:
        gdf_csd_small_proj = gdf_csd_small.to_crs(3857)
    else:
        # For years 1961-1980, use 2021 CSD geography with NaN values
        gdf_csd_small_proj = gdf_csd_gta_2021.to_crs(3857)
    
    # Create a unified geometry of all CTs to identify non-overlapping CSD areas
    ct_union = gdf_ct_small_proj.unary_union
    
    # Calculate the difference between CSD geometries and CT union
    gdf_csd_non_overlap = gdf_csd_small_proj.copy()
    gdf_csd_non_overlap['geometry'] = gdf_csd_non_overlap.geometry.difference(ct_union)
    
    # Calculate area ratio for weighting census variables
    original_areas = gdf_csd_small_proj.geometry.area
    new_areas = gdf_csd_non_overlap.geometry.area
    area_ratios = new_areas / original_areas

    # Remove minor differences in the geometry 
    area_ratios[area_ratios < 0.02] = 0
    gdf_csd_non_overlap.loc[area_ratios == 0, 'geometry'] = Polygon()
    
    # Weight census variables by area ratio (only for years with actual CSD data)
    if cen_year >= 1981:
        census_cols = [col for col in gdf_csd_small.columns if col not in ['geosid', 'geometry']]
        for col in census_cols:
            gdf_csd_non_overlap[col] = gdf_csd_small[col] * area_ratios
    
    # Instead of filtering out empty geometries, keep them but set census values to NaN
    empty_mask = gdf_csd_non_overlap.is_empty
    if empty_mask.any():
        census_cols = [col for col in gdf_csd_non_overlap.columns if col not in ['geosid', 'geometry']]
        gdf_csd_non_overlap.loc[empty_mask, census_cols] = np.nan
    
    # Combine CTs and non-overlapping CSDs
    gdf_combined = pd.concat([gdf_ct_small_proj, gdf_csd_non_overlap], ignore_index=True)
    
    # Ensure valid geometries and convert back to original CRS
    invalid_mask = ~gdf_combined.is_valid
    if invalid_mask.any():
        gdf_combined.loc[invalid_mask, 'geometry'] = gdf_combined[invalid_mask].buffer(0)
        census_cols = [col for col in gdf_combined.columns if col not in ['geosid', 'geometry']]
        gdf_combined.loc[invalid_mask, census_cols] = np.nan
    
    gdf_combined = gdf_combined.to_crs(gdf_ct_small.crs)
    
    # Clean data (set any negative values to NaN)
    numeric_cols = ['num_imm_tot', 'num_pop_tot', 'num_imm_new', 'num_imm_2nd_tot', 'num_vm_tot']
    for col in numeric_cols:
        if col in gdf_combined.columns:
            gdf_combined[col] = pd.to_numeric(gdf_combined[col], errors='coerce')
            if col == 'num_pop_tot':
                gdf_combined.loc[gdf_combined[col] <= 0, col] = np.nan
            else:
                gdf_combined.loc[gdf_combined[col] < 0, col] = np.nan
    
    # Calculate percentage of immigrants
    gdf_combined['pct_imm'] = (gdf_combined['num_imm_tot'] / gdf_combined['num_pop_tot']) * 100
    gdf_combined['pct_imm_new'] = (gdf_combined['num_imm_new'] / gdf_combined['num_pop_tot']) * 100
    gdf_combined['pct_imm_2nd'] = (gdf_combined['num_imm_2nd_tot'] / gdf_combined['num_pop_tot']) * 100
    gdf_combined['pct_vm'] = (gdf_combined['num_vm_tot'] / gdf_combined['num_pop_tot']) * 100
    
    # Safer coverage gap check
    try:
        current_coverage = gdf_combined.unary_union
        original_coverage = gdf_csd_small_proj.to_crs(gdf_ct_small.crs).unary_union if cen_year >= 1981 else gdf_csd_gta_2021.unary_union
        coverage_gap = original_coverage.difference(current_coverage)
        
        if not coverage_gap.is_empty and coverage_gap.area > 1:  # 1 square meter threshold
            gap_row = {
                'geosid': 'MISSING',
                'geometry': coverage_gap,
                'num_pop_tot': np.nan,
                'num_imm_tot': np.nan,
                'pct_imm': np.nan
            }
            for col in gdf_combined.columns:
                if col not in gap_row:
                    gap_row[col] = np.nan
            
            gap_gdf = gpd.GeoDataFrame([gap_row], crs=gdf_combined.crs)
            gdf_combined = pd.concat([gdf_combined, gap_gdf], ignore_index=True)
    except:
        pass
    
    # Save the final output
    gdf_combined.to_file(f"../data/immigration/{cen_year}/imm_stats_{cen_year}.gpkg", driver="GPKG")

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


In [ ]:
# Manual cleanup of clipping using Lake Simcoe and Lake Scugog first, then running this
for cen_year in tqdm(CEN_YEARS):
    if (cen_year < 1961) or (cen_year == 1966) or (cen_year == 1976):
        continue

    gdf_imm_stats = gpd.read_file(f"../data/immigration/{cen_year}/imm_stats_{cen_year}.gpkg")
    gdf_imm_stats.to_file(f'../static/data/immigration/imm_stats_{cen_year}.geojson')

100%|██████████| 15/15 [00:04<00:00,  3.34it/s]
